In [2]:
!pip install selenium webdriver_manager beautifulsoup4

In [3]:
import re
import io
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

**Código apenas print da página**

In [5]:

# Configura o navegador para rodar em modo headless (sem interface gráfica)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Executa sem abrir a janela
chrome_options.add_argument("--disable-gpu")  # Desativa a aceleração gráfica (útil em alguns ambientes)
chrome_options.add_argument("--window-size=1920,1080")  # Define o tamanho da janela para evitar problemas de renderização
chrome_options.add_argument("--no-sandbox")  # Configuração útil em alguns ambientes Linux

# Inicializa o driver usando o ChromeDriver gerenciado automaticamente
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # 1. Abre a página do Minor Planet Center
    driver.get("https://minorplanetcenter.net/iau/NEO/pccp_tabular.html")
    
    # 2. Localiza o radio button <input type="radio" name="W" value="a">, rola a página até ele e clica via JavaScript
    radio_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//input[@type='radio' and @name='W' and @value='a']"))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", radio_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", radio_button)
    
    # 3. Localiza o campo do código do observatório e insere "Y28"
    obs_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@name='obscode']"))
    )
    obs_input.clear()
    obs_input.send_keys("Y28")
    
    # 4. Localiza e clica no botão de submissão via JavaScript
    submit_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@type='submit']"))
    )
    driver.execute_script("arguments[0].click();", submit_button)
    
    # 5. Aguarda alguns segundos para a página carregar
    time.sleep(5)
    
    # 6. Extrai e imprime todo o texto da página
    body_element = driver.find_element(By.TAG_NAME, "body")
    page_text = body_element.text
    print(page_text)
    
finally:
    driver.quit()

Quick links : Home Page : Contact Us : Index : Site Map : Search Site
NEO Confirmation Page: Query Results
Below are the results of your request from the Minor Planet Center's NEO Confirmation Page.
Use the feedback form to report problems or to comment on this page.
Ephemerides are for observatory code Y28.
5FI3921
Get the observations or orbits.
Date       UT   *  R.A. (J2000) Decl.  Elong.  V        Motion     Object     Sun         Moon        Uncertainty
            h                                        "/min   P.A.  Azi. Alt.  Alt.  Phase Dist. Alt.
... <suppressed> ...
2025 06 20 19     14 28 31.9 -28 27 34 133.2  18.1   45.04  254.9  298  +29   +18    0.28  159  -44   Map/Offsets   !
2025 06 20 20     14 25 07.0 -28 39 19 132.4  18.0   47.97  255.8  301  +43   +04    0.28  160  -58   Map/Offsets   !
2025 06 20 21     14 21 27.9 -28 51 03 131.6  18.0   50.92  256.7  310  +56   -09    0.27  161  -72   Map/Offsets   !
2025 06 20 22     14 17 34.3 -29 02 38 130.8  18.0   53.84  

**Código obtem e cria tabela pandas**

In [4]:
def parse_extended_line(line):
    tokens = line.split()
    if len(tokens) < 20:
        return None
    
    date_val      = " ".join(tokens[0:3])         
    ut_val        = tokens[3]                    
    ra_val        = " ".join(tokens[4:7])         
    decl_val      = " ".join(tokens[7:10])        
    elong_val     = tokens[10]                    
    v_val         = tokens[11]                    
    motion_min    = tokens[12]                   
    motion_PA     = tokens[13]
    obj_azi       = tokens[14]                    
    obj_alt       = tokens[15]                    
    sun_alt       = tokens[16]                    
    moon_phase    = tokens[17]                    
    moon_dist     = tokens[18]                    
    moon_alt      = tokens[19]                    
                       
    return [date_val, ut_val, ra_val, decl_val, elong_val, v_val,
            motion_min, motion_PA, obj_azi, obj_alt, sun_alt, moon_phase, moon_dist,
            moon_alt]



chrome_options = Options()
chrome_options.add_argument("--headless")          # executa sem interface gráfica
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    driver.get("https://minorplanetcenter.net/iau/NEO/pccp_tabular.html")
    
    
    radio_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//input[@type='radio' and @name='W' and @value='a']"))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", radio_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", radio_button)
    
    
    obs_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@name='obscode']"))
    )
    obs_input.clear()
    obs_input.send_keys("Y28")
    
    
    submit_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@type='submit']"))
    )
    driver.execute_script("arguments[0].click();", submit_button)
    
    
    time.sleep(5)
    
    
    body_element = driver.find_element(By.TAG_NAME, "body")
    page_text = body_element.text
    #print("Texto completo extraído:")
    #print(page_text)
    
finally:
    driver.quit()


lines = page_text.splitlines()
blocks = {}  # dicionário: {nome_objeto: [linhas do bloco]}
current_object = None
current_block = []

for i, line in enumerate(lines):
    line_stripped = line.strip()
    
    if line_stripped and (i + 1 < len(lines)) and ("Get the observations" in lines[i+1]):
        if current_object and current_block:
            blocks[current_object] = current_block
        current_object = line_stripped
        current_block = []
        continue
    if current_object:
        
        if "Get the observations" in line_stripped:
            continue
        if line_stripped:
            current_block.append(line_stripped)
if current_object and current_block:
    blocks[current_object] = current_block


column_names = [
    "Date",
    "UT",
    "R.A. (J2000)",
    "Decl",
    "Elong",
    "V",
    'Motion min',
    'Motion PA',
    "Object Azi",
    "Object Alt",
    "Sun Alt",
    "Moon Phase",
    "Moon Dist",
    "Moon Alt"
]


dataframes = {}
for obj, block_lines in blocks.items():
    rows = []
    for line in block_lines:
        # Seleciona linhas que começam com 4 dígitos (supõe que iniciam com o ano)
        if re.match(r'^\d{4}', line.strip()):
            parsed = parse_extended_line(line)
            if parsed is not None:
                rows.append(parsed)
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        dataframes[obj] = df

# Exibe os DataFrames resultantes para cada objeto
for obj, df in dataframes.items():
    print(f"\nDataFrame para o objeto {obj}:")
    print(df)


DataFrame para o objeto 5FI3921:
         Date  UT R.A. (J2000)       Decl  Elong     V Motion min Motion PA  \
0  2025 06 20  19   14 28 31.9  -28 27 34  133.2  18.1      45.04     254.9   
1  2025 06 20  20   14 25 07.0  -28 39 19  132.4  18.0      47.97     255.8   
2  2025 06 20  21   14 21 27.9  -28 51 03  131.6  18.0      50.92     256.7   
3  2025 06 20  22   14 17 34.3  -29 02 38  130.8  18.0      53.84     257.7   
4  2025 06 20  23   14 13 26.5  -29 13 56  129.9  17.9      56.72     258.7   
5  2025 06 21  00   14 09 04.6  -29 24 53  128.9  17.9      59.51     259.6   
6  2025 06 21  01   14 04 29.0  -29 35 24  127.9  17.9      62.24     260.5   
7  2025 06 21  02   13 59 40.0  -29 45 27  126.8  17.8      64.91     261.3   
8  2025 06 21  03   13 54 37.8  -29 55 02  125.7  17.8      67.57     262.0   

  Object Azi Object Alt Sun Alt Moon Phase Moon Dist Moon Alt  
0        298        +29     +18       0.28       159      -44  
1        301        +43     +04       0.28     

**Teste de utilização**

In [6]:
df = dataframes["5FI3921"]


valor = df.iloc[8]['Decl']
print(valor) 

-29 55 02
